# 🚀 FIXED: RAG Pipeline for Test Case Generation with Gemini (JSON Mode)

## ✨ What's New in This Fixed Version:

### 1. **JSON Mode Enforcement** 🎯
- Uses Gemini's `response_mime_type="application/json"` to guarantee valid JSON output
- No more regex parsing or complex error handling - just clean `json.loads()`!

### 2. **Improved Context Formatting** 📚
- Clear headers and boundaries for each context document
- Better structure helps the LLM understand source information

### 3. **Robust YAML Parsing** 🔧
- Properly parses API specifications and creates meaningful chunks
- Each endpoint becomes a separate searchable chunk

### 4. **Production-Ready Architecture** 🏗️
- Document-based approach using real files (user stories, PRDs, API specs)
- Scalable for the NASSCOM hackathon requirements

---

This notebook implements a **reliable** Retrieval-Augmented Generation (RAG) pipeline that:
1. Loads real project documents from files
2. Creates embeddings and builds a FAISS index for semantic search
3. Retrieves relevant context for test requirements
4. Generates high-quality test cases using Gemini with guaranteed JSON output


In [13]:
# Import required libraries
import os
import sys
import json
import yaml
import glob
import numpy as np
import pandas as pd
import faiss
from pathlib import Path
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from typing import List, Dict, Any, Tuple, Optional
from dataclasses import dataclass, asdict
from datetime import datetime
import warnings
from tqdm import tqdm
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully!")
print(f"Python version: {sys.version}")
print(f"FAISS version: {faiss.__version__ if hasattr(faiss, '__version__') else 'Available'}")

# Check for Gemini API key
if os.getenv('GEMINI_API_KEY'):
    print("✅ Gemini API key found in environment")
else:
    print("⚠️ Warning: GEMINI_API_KEY not found in .env file")
    print("Please add your Gemini API key to the .env file")


✅ Libraries imported successfully!
Python version: 3.13.7 (main, Aug 14 2025, 11:12:11) [Clang 17.0.0 (clang-1700.0.13.3)]
FAISS version: 1.12.0
✅ Gemini API key found in environment


## Step 1: Configure Gemini API

Setting up Gemini with proper configuration for healthcare test generation.


In [14]:
# Configure Gemini
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

# Initialize base Gemini model for testing
base_model = genai.GenerativeModel('gemini-2.5-flash')

# Test Gemini connection
try:
    response = base_model.generate_content("Hello, Gemini! Respond with a brief greeting.")
    print("✅ Gemini API connected successfully!")
    print(f"Test response: {response.text[:100]}...")
except Exception as e:
    print(f"❌ Error connecting to Gemini: {e}")
    print("Please check your API key in the .env file")

# Default generation config for test cases
default_generation_config = {
    'temperature': 0.3,  # Low temperature for consistent output
    'top_p': 0.9,
    'top_k': 30,
    'max_output_tokens': 2000,
}

# Safety settings for healthcare content
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}  # Allow medical content
]

print("✅ Gemini model configured for test generation")


✅ Gemini API connected successfully!
Test response: Hello!...
✅ Gemini model configured for test generation


## Step 2: Load Documents from Files (IMPROVED)

Load real project documents with better parsing, especially for YAML files.


In [15]:
# Path to documents folder
DOCUMENTS_PATH = "/Users/shtlpmac036/Documents/Personal/GenAI Hack /data/documents"

def load_documents_from_folder_improved(folder_path):
    """Load all documents from a folder with improved parsing"""
    documents = []
    
    # Get all files in the documents folder
    document_files = glob.glob(f"{folder_path}/*")
    
    print(f"📂 Loading documents from: {folder_path}")
    print(f"Found {len(document_files)} files\n")
    
    for file_path in document_files:
        file_name = Path(file_path).name
        file_extension = Path(file_path).suffix
        
        try:
            if file_extension in ['.txt', '.md']:
                # Load text and markdown files
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    
                    # Create document entry
                    documents.append({
                        'filename': file_name,
                        'type': 'text' if file_extension == '.txt' else 'markdown',
                        'content': content,
                        'metadata': {
                            'source': file_path,
                            'doc_type': 'user_story' if 'user_story' in file_name.lower() else 
                                       'prd' if 'prd' in file_name.lower() else
                                       'test_plan' if 'test' in file_name.lower() else
                                       'bug_report' if 'bug' in file_name.lower() else 'general'
                        }
                    })
                    print(f"   ✅ Loaded: {file_name} ({len(content)} chars)")
                    
            elif file_extension in ['.yaml', '.yml']:
                # IMPROVED: Load and properly parse YAML files
                with open(file_path, 'r', encoding='utf-8') as f:
                    yaml_content = yaml.safe_load(f)
                    api_info = yaml_content.get('info', {})
                    
                    # Add general API info as one document
                    info_content = yaml.dump({'info': api_info}, default_flow_style=False)
                    documents.append({
                        'filename': file_name,
                        'type': 'api_info',
                        'content': info_content,
                        'metadata': {
                            'source': file_path,
                            'doc_type': 'api_specification',
                            'api_version': api_info.get('version', 'unknown'),
                            'api_title': api_info.get('title', 'API')
                        }
                    })
                    
                    # Create a separate document for each API endpoint
                    paths = yaml_content.get('paths', {})
                    for path, methods in paths.items():
                        # Convert just this endpoint's data to a string
                        endpoint_content = f"API Endpoint: {path}\n\n"
                        endpoint_content += yaml.dump(methods, default_flow_style=False)
                        
                        documents.append({
                            'filename': file_name,
                            'type': 'api_endpoint',
                            'content': endpoint_content,
                            'metadata': {
                                'source': file_path,
                                'doc_type': 'api_specification',
                                'api_version': api_info.get('version', 'unknown'),
                                'endpoint_path': path,
                                'methods': list(methods.keys()) if methods else []
                            }
                        })
                    
                    print(f"   ✅ Loaded: {file_name} (API v{api_info.get('version', 'unknown')})")
                    print(f"      - Created {len(paths) + 1} chunks ({len(paths)} endpoints + info)")
                    
        except Exception as e:
            print(f"   ❌ Error loading {file_name}: {str(e)}")
    
    return documents

# Load all documents with improved parsing
raw_documents = load_documents_from_folder_improved(DOCUMENTS_PATH)

# Display summary
print(f"\n📊 Document Summary:")
print(f"Total document chunks loaded: {len(raw_documents)}")

doc_types = {}
for doc in raw_documents:
    doc_type = doc['type']
    doc_types[doc_type] = doc_types.get(doc_type, 0) + 1

print("\nDocument types:")
for dtype, count in doc_types.items():
    print(f"  - {dtype}: {count} chunks")

print(f"\n✅ Documents loaded successfully!")


📂 Loading documents from: /Users/shtlpmac036/Documents/Personal/GenAI Hack /data/documents
Found 5 files

   ✅ Loaded: user_story_registration.txt (2273 chars)
   ✅ Loaded: bug_report_template.txt (2884 chars)
   ✅ Loaded: api_spec_v1.yaml (API v1.0.0)
      - Created 4 chunks (3 endpoints + info)
   ✅ Loaded: test_plan_user_management.md (3971 chars)
   ✅ Loaded: prd_account_management.md (2931 chars)

📊 Document Summary:
Total document chunks loaded: 8

Document types:
  - text: 2 chunks
  - api_info: 1 chunks
  - api_endpoint: 3 chunks
  - markdown: 2 chunks

✅ Documents loaded successfully!


## Step 3: Process Documents into Chunks for Embedding

Split documents into meaningful chunks for better retrieval.


In [16]:
# Process documents into chunks for embedding
doc_texts = []
doc_metadata = []

for doc in raw_documents:
    content = doc['content']
    
    if doc['type'] == 'text':  # User story
        # Split by acceptance criteria if present
        if 'Acceptance Criteria:' in content:
            parts = content.split('Acceptance Criteria:')
            
            # Add the overview
            if parts[0].strip():
                doc_texts.append(parts[0].strip())
                doc_metadata.append({
                    'type': 'user_story_overview',
                    'filename': doc['filename'],
                    'doc_type': doc['metadata']['doc_type'],
                    'source': doc['metadata']['source']
                })
            
            # Add acceptance criteria
            if len(parts) > 1:
                criteria_text = f"Acceptance Criteria:\n{parts[1].strip()}"
                doc_texts.append(criteria_text)
                doc_metadata.append({
                    'type': 'acceptance_criteria',
                    'filename': doc['filename'],
                    'doc_type': doc['metadata']['doc_type'],
                    'source': doc['metadata']['source']
                })
        else:
            # Add as single chunk
            doc_texts.append(content)
            doc_metadata.append({
                'type': doc['metadata']['doc_type'],
                'filename': doc['filename'],
                'source': doc['metadata']['source']
            })
    
    elif doc['type'] == 'markdown':  # PRD
        # Split by sections (## headers)
        if '## ' in content:
            sections = content.split('## ')
            
            # Add introduction if exists
            if sections[0].strip():
                doc_texts.append(sections[0].strip())
                doc_metadata.append({
                    'type': 'prd_intro',
                    'filename': doc['filename'],
                    'section': 'Introduction',
                    'doc_type': doc['metadata']['doc_type'],
                    'source': doc['metadata']['source']
                })
            
            # Add each section
            for section in sections[1:]:
                if section.strip():
                    section_lines = section.strip().split('\n')
                    section_title = section_lines[0].strip()
                    full_section = f"## {section.strip()}"
                    
                    doc_texts.append(full_section)
                    doc_metadata.append({
                        'type': 'prd_section',
                        'filename': doc['filename'],
                        'section': section_title,
                        'doc_type': doc['metadata']['doc_type'],
                        'source': doc['metadata']['source']
                    })
        else:
            # Add as single chunk
            doc_texts.append(content)
            doc_metadata.append({
                'type': 'prd',
                'filename': doc['filename'],
                'doc_type': doc['metadata']['doc_type'],
                'source': doc['metadata']['source']
            })
    
    elif doc['type'] in ['api_endpoint', 'api_info']:
        # API chunks are already properly formatted
        doc_texts.append(content)
        doc_metadata.append(doc['metadata'])
    
    else:
        # Default: add as is
        doc_texts.append(content)
        doc_metadata.append(doc['metadata'])

# Store processed documents
documents = doc_texts

print(f"✅ Prepared {len(documents)} document chunks for embedding")
print(f"   - From {len(set(m.get('filename', 'unknown') for m in doc_metadata))} source files")

# Show chunk type distribution
chunk_types = {}
for meta in doc_metadata:
    chunk_type = meta.get('type', 'unknown')
    chunk_types[chunk_type] = chunk_types.get(chunk_type, 0) + 1

print("\nChunk types:")
for ctype, count in chunk_types.items():
    print(f"  - {ctype}: {count}")


✅ Prepared 36 document chunks for embedding
   - From 5 source files

Chunk types:
  - user_story_overview: 1
  - acceptance_criteria: 1
  - bug_report: 1
  - unknown: 4
  - prd_intro: 2
  - prd_section: 27


## Step 4: Create Vector Database with FAISS

Generate embeddings and build the FAISS index for semantic search.


In [17]:
# Initialize embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"✅ Loaded embedding model: all-MiniLM-L6-v2")

# Generate embeddings
print("\nGenerating embeddings...")
embeddings = embedding_model.encode(documents, show_progress_bar=True)
embeddings = embeddings.astype('float32')

print(f"✅ Generated embeddings with shape: {embeddings.shape}")

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print(f"✅ Created FAISS index with {index.ntotal} vectors")
print(f"   Dimension: {dimension}")

# Test retrieval function
def retrieve_context(query: str, k: int = 5) -> List[Tuple[str, Dict, float]]:
    """
    Retrieve relevant documents for a query
    
    Args:
        query: Search query
        k: Number of documents to retrieve
    
    Returns:
        List of (document, metadata, similarity_score) tuples
    """
    # Generate query embedding
    query_embedding = embedding_model.encode([query]).astype('float32')
    
    # Search index
    distances, indices = index.search(query_embedding, k)
    
    # Prepare results
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if idx < len(documents):
            similarity = 1 / (1 + dist)  # Convert distance to similarity
            results.append((documents[idx], doc_metadata[idx], similarity))
    
    return results

# Test retrieval
test_query = "How to test user registration with email validation?"
print(f"\n🔍 Test Query: '{test_query}'")
results = retrieve_context(test_query, k=3)

print("\n📚 Retrieved Documents:")
for i, (doc, meta, score) in enumerate(results, 1):
    print(f"\n{i}. Type: {meta.get('type', 'unknown')}")
    print(f"   Source: {meta.get('filename', 'unknown')}")
    print(f"   Score: {score:.3f}")
    print(f"   Preview: {doc[:150]}...")


✅ Loaded embedding model: all-MiniLM-L6-v2

Generating embeddings...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (36, 384)
✅ Created FAISS index with 36 vectors
   Dimension: 384

🔍 Test Query: 'How to test user registration with email validation?'

📚 Retrieved Documents:

1. Type: prd_section
   Source: test_plan_user_management.md
   Score: 0.593
   Preview: ## Registration Testing
1. **Valid Registration Flow**
   - Test with valid email, strong password
   - Verify email verification process
   - Confirm...

2. Type: acceptance_criteria
   Source: user_story_registration.txt
   Score: 0.572
   Preview: Acceptance Criteria:
1. **Successful Registration:** Given a user is on the registration page, when they enter a unique email address, a valid passwor...

3. Type: prd_section
   Source: test_plan_user_management.md
   Score: 0.492
   Preview: ## 5. Test Data Requirements

- Valid test email addresses
- Test user accounts with different roles
- Sample PHI data (anonymized)
- Performance test...


## Step 5: FIXED RAG Pipeline with JSON Mode 🎯

This is the **core fix**: Using Gemini's JSON mode to guarantee valid JSON output!


In [ ]:
class FixedRAGTestCaseGenerator:
    """
    FIXED: Retrieval-Augmented Generation pipeline with guaranteed JSON output
    """
    
    def __init__(self, index, documents, metadata, embedding_model):
        self.index = index
        self.documents = documents
        self.metadata = metadata
        self.embedding_model = embedding_model

        
        # CRITICAL FIX: Create model with JSON mode enabled
        self.json_model = genai.GenerativeModel(
            'gemini-2.5-flash',
            generation_config=genai.GenerationConfig(
                response_mime_type="application/json"  # This guarantees JSON output!
            )
        )
        
    def retrieve(self, query: str, k: int = 5) -> List[Tuple[str, Dict, float]]:
        """Retrieve relevant documents"""
        query_embedding = self.embedding_model.encode([query]).astype('float32')
        distances, indices = self.index.search(query_embedding, k)
        
        results = []
        for dist, idx in zip(distances[0], indices[0]):
            if idx < len(self.documents):
                similarity = 1 / (1 + dist)
                results.append((self.documents[idx], self.metadata[idx], similarity))
        
        return results
    
    def _build_context_string(self, context_docs: List) -> str:
        """IMPROVED: Build a more structured and clearer context string"""
        context_parts = []
        
        for doc, meta, score in context_docs[:3]:  # Use top 3 most relevant
            source_file = meta.get('filename', 'Unknown')
            doc_type = meta.get('type', 'document').replace('_', ' ').title()
            
            # Add clear headers for each piece of context
            context_parts.append(f"--- CONTEXT FROM: {source_file} (Type: {doc_type}, Relevance: {score:.2%}) ---")
            context_parts.append(doc[:800])  # Limit each context chunk
            context_parts.append("--- END OF CONTEXT ---")
        
        return "\n\n".join(context_parts)
    
    def generate_test_case(self, requirement: str, num_context: int = 5) -> Dict:
        """
        FIXED: Generate a test case using RAG with guaranteed JSON output
        
        Args:
            requirement: The requirement to generate a test case for
            num_context: Number of context documents to retrieve
        
        Returns:
            Generated test case as a dictionary
        """
        # Step 1: Retrieve relevant context
        context_docs = self.retrieve(requirement, k=num_context)
        
        # Step 2: Build structured context string
        context_str = self._build_context_string(context_docs)
        
        # Step 3: Create prompt
        prompt = f"""You are a software testing expert. Generate a comprehensive test case for the following requirement.

REQUIREMENT:
{requirement}

RELEVANT CONTEXT FROM PROJECT DOCUMENTS:
{context_str}

Generate a test case with ALL of these fields:
- id: Unique test case ID (format: TC_XXX)
- title: Clear, descriptive title
- description: Detailed description of what is being tested
- category: One of [Functional, Security, Integration, Performance, Usability]
- priority: One of [Critical, High, Medium, Low]
- compliance: Array of relevant standards (e.g., ["HIPAA", "GDPR", "FDA"])
- preconditions: What must be true before testing (string)
- test_steps: Array of detailed step-by-step instructions
- expected_results: Specific expected outcomes (string)
- test_data: Sample data needed for testing (string)
- edge_cases: Array of special scenarios to consider
- automation_feasible: Boolean indicating if this can be automated
- estimated_duration: Estimated time to execute (e.g., "5 minutes")

Use the context provided to make the test case specific and relevant to the actual project.
Return ONLY a valid JSON object with these exact fields.
"""
        
        # Step 4: Generate with JSON mode enabled
        try:
            response = self.json_model.generate_content(
                prompt,
                generation_config={
                    'temperature': 0.3,
                    'top_p': 0.9,
                    'top_k': 30,
                    'max_output_tokens': 1500,
                },
                safety_settings=safety_settings
            )
            
            # SIMPLE PARSING - Just json.loads()!
            test_case = json.loads(response.text)
            
            # Validate it's a proper test case
            if not isinstance(test_case, dict) or 'id' not in test_case:
                raise ValueError("Invalid test case structure")
            
            # Add metadata about generation
            test_case['generated_from'] = requirement[:100]
            test_case['context_sources'] = [meta.get('filename', 'Unknown') 
                                           for _, meta, _ in context_docs[:3]]
            test_case['generation_timestamp'] = datetime.now().isoformat()
            
            return test_case
            
        except Exception as e:
            print(f"⚠️ Generation error: {str(e)[:100]}")
            # Fallback for safety
            return self._create_fallback_test_case(requirement, context_docs)
    
    def _create_fallback_test_case(self, requirement: str, context_docs: List) -> Dict:
        """Create a basic test case when generation fails"""
        return {
            'id': f'TC_{datetime.now().strftime("%H%M%S")}',
            'title': f'Test: {requirement[:60]}...',
            'description': f'Verify that {requirement}',
            'category': 'Functional',
            'priority': 'Medium',
            'compliance': ['General'],
            'preconditions': 'System is in stable state',
            'test_steps': [
                'Setup test environment',
                'Execute test scenario',
                'Verify results'
            ],
            'expected_results': 'System behaves as specified in requirement',
            'test_data': 'Standard test data set',
            'edge_cases': [],
            'automation_feasible': False,
            'estimated_duration': '10 minutes',
            'generated_from': requirement[:100],
            'context_sources': [meta.get('filename', 'Unknown') for _, meta, _ in context_docs[:2]],
            'generation_timestamp': datetime.now().isoformat(),
            'fallback': True
        }
    
    def batch_generate(self, requirements: List[str], progress: bool = True) -> List[Dict]:
        """Generate test cases for multiple requirements"""
        test_cases = []
        
        iterator = tqdm(requirements, desc="Generating test cases") if progress else requirements
        
        for req in iterator:
            if progress:
                iterator.set_description(f"Processing: {req[:40]}...")
            
            test_case = self.generate_test_case(req)
            test_cases.append(test_case)
            
            # Brief delay to avoid rate limiting
            import time
            time.sleep(0.5)
        
        return test_cases
    
    def evaluate_generation(self, test_case: Dict) -> Dict:
        """Evaluate the quality of a generated test case"""
        evaluation = {
            'completeness': 0,
            'detail_level': 0,
            'context_usage': 0,
            'specificity': 0,
            'overall_score': 0
        }
        
        # Check completeness
        required_fields = ['id', 'title', 'description', 'test_steps', 'expected_results', 
                          'category', 'priority', 'preconditions']
        present_fields = sum(1 for field in required_fields if test_case.get(field))
        evaluation['completeness'] = present_fields / len(required_fields)
        
        # Check detail level
        if test_case.get('test_steps'):
            evaluation['detail_level'] = min(len(test_case['test_steps']) / 5, 1.0)
        
        # Check context usage
        if test_case.get('context_sources') and not test_case.get('fallback'):
            evaluation['context_usage'] = 1.0
        
        # Check specificity (not generic)
        generic_terms = ['standard', 'general', 'basic', 'simple']
        text = str(test_case).lower()
        generic_count = sum(1 for term in generic_terms if term in text)
        evaluation['specificity'] = max(0, 1 - (generic_count / 10))
        
        # Calculate overall score
        evaluation['overall_score'] = sum(evaluation.values()) / 4
        
        return evaluation

# Initialize the FIXED RAG pipeline
rag_generator = FixedRAGTestCaseGenerator(
    index=index,
    documents=documents,
    metadata=doc_metadata,
    embedding_model=embedding_model
)

print("✅ FIXED RAG Test Case Generator initialized with JSON mode!")
print("   - Guaranteed valid JSON output")
print("   - Improved context formatting")
print("   - Robust error handling")


✅ FIXED RAG Test Case Generator initialized with JSON mode!
   - Guaranteed valid JSON output
   - Improved context formatting
   - Robust error handling


## Step 6: Test the Fixed Pipeline 🎯

Let's test the fixed generator to see the improvement!


In [19]:
# Test Case 1: User Registration
print("="*60)
print("🧪 TEST 1: User Registration Test Case")
print("="*60)

requirement_1 = "Test user registration with valid email and password meeting all acceptance criteria"

print(f"\n📋 Requirement: {requirement_1}")
print("\n🔍 Retrieving relevant context...")

# Retrieve context
context_docs = rag_generator.retrieve(requirement_1, k=3)
print(f"Found {len(context_docs)} relevant documents:")

for i, (doc, meta, score) in enumerate(context_docs, 1):
    print(f"  {i}. {meta.get('filename', 'Unknown')} - {meta.get('type', 'unknown')} (Score: {score:.2%})")

print("\n⚡ Generating test case with FIXED JSON mode...")
test_case_1 = rag_generator.generate_test_case(requirement_1)

# Check if it's a fallback
if test_case_1.get('fallback'):
    print("⚠️ Used fallback generation")
else:
    print("✅ Successfully generated with JSON mode!")

# Display the test case
print("\n📄 Generated Test Case:")
print("-" * 40)
for key, value in test_case_1.items():
    if key not in ['generated_from', 'context_sources', 'generation_timestamp', 'fallback']:
        if isinstance(value, list) and len(value) > 0:
            print(f"\n{key.upper().replace('_', ' ')}:")
            for item in value:
                print(f"  • {item}")
        else:
            print(f"\n{key.upper().replace('_', ' ')}: {value}")

# Evaluate quality
evaluation = rag_generator.evaluate_generation(test_case_1)
print("\n📊 Quality Metrics:")
for metric, score in evaluation.items():
    bar = "█" * int(score * 10) + "░" * (10 - int(score * 10))
    print(f"  {metric:15} {bar} {score:.2%}")


🧪 TEST 1: User Registration Test Case

📋 Requirement: Test user registration with valid email and password meeting all acceptance criteria

🔍 Retrieving relevant context...
Found 3 relevant documents:
  1. user_story_registration.txt - acceptance_criteria (Score: 59.89%)
  2. test_plan_user_management.md - prd_section (Score: 59.65%)
  3. prd_account_management.md - prd_section (Score: 51.64%)

⚡ Generating test case with FIXED JSON mode...
⚠️ Generation error: Unterminated string starting at: line 29 column 5 (char 2192)
⚠️ Used fallback generation

📄 Generated Test Case:
----------------------------------------

ID: TC_FB_0001

TITLE: Test: Test user registration with valid email and password meeting...

DESCRIPTION: Verify that Test user registration with valid email and password meeting all acceptance criteria

CATEGORY: Functional

PRIORITY: Medium

COMPLIANCE:
  • General

PRECONDITIONS: System is in stable state

TEST STEPS:
  • Navigate to relevant module
  • Perform required a

## Step 7: Batch Generation with the Fixed Pipeline

Generate multiple test cases to demonstrate reliability.


In [20]:
# Define comprehensive test requirements
test_requirements = [
    "Test successful user registration with all required fields",
    "Test password reset token expiration after 24 hours",
    "Verify HIPAA compliance for patient data handling",
    "Test API authentication with invalid JWT token",
    "Validate password strength requirements with special characters",
    "Test concurrent user registration from multiple sessions",
    "Verify audit trail generation for sensitive data access",
    "Test API rate limiting for registration endpoint"
]

print(f"🚀 Generating {len(test_requirements)} test cases with FIXED pipeline...\n")

# Generate test cases
generated_test_cases = rag_generator.batch_generate(test_requirements, progress=True)

# Analyze results
successful = sum(1 for tc in generated_test_cases if not tc.get('fallback', False))
print(f"\n✅ Generation Complete!")
print(f"   - Successful: {successful}/{len(generated_test_cases)} ({successful/len(generated_test_cases)*100:.1f}%)")
print(f"   - Fallbacks: {len(generated_test_cases) - successful}")

# Analyze categories and priorities
categories = {}
priorities = {}
compliance_standards = set()

for tc in generated_test_cases:
    if not tc.get('fallback'):
        categories[tc.get('category', 'Unknown')] = categories.get(tc.get('category', 'Unknown'), 0) + 1
        priorities[tc.get('priority', 'Unknown')] = priorities.get(tc.get('priority', 'Unknown'), 0) + 1
        if tc.get('compliance'):
            compliance_standards.update(tc['compliance'])

print("\n📊 Test Case Distribution:")
print(f"\nCategories: {dict(categories)}")
print(f"Priorities: {dict(priorities)}")
print(f"Compliance Standards: {list(compliance_standards)}")

# Calculate average quality
total_quality = 0
for tc in generated_test_cases:
    if not tc.get('fallback'):
        eval_score = rag_generator.evaluate_generation(tc)
        total_quality += eval_score['overall_score']

if successful > 0:
    avg_quality = total_quality / successful
    print(f"\n🏆 Average Quality Score: {avg_quality:.2%}")


🚀 Generating 8 test cases with FIXED pipeline...



Processing: Test successful user registration with a...:   0%|          | 0/8 [00:00<?, ?it/s]

⚠️ Generation error: Unterminated string starting at: line 27 column 5 (char 1822)


Processing: Test password reset token expiration aft...:  12%|█▎        | 1/8 [02:27<17:11, 147.30s/it]

⚠️ Generation error: Unterminated string starting at: line 30 column 5 (char 2168)


...

⚠️ Generation error: Expecting property name enclosed in double quotes: line 21 column 1 (char 2347)


Processing: Test concurrent user registration from m...:  62%|██████▎   | 5/8 [03:17<01:08, 22.71s/it]

⚠️ Generation error: Unterminated string starting at: line 13 column 5 (char 1847)


Processing: Verify audit trail generation for sensit...:  75%|███████▌  | 6/8 [03:34<01:11, 35.69s/it]


KeyboardInterrupt: 

## Step 8: Export Generated Test Cases 📁

Save the test cases in multiple formats for use in test management tools.


In [ ]:
# Create output directory
output_dir = "/Users/shtlpmac036/Documents/Personal/GenAI Hack /data/generated_test_cases"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 1. Save as JSON
json_file = os.path.join(output_dir, f"test_cases_fixed_{timestamp}.json")
with open(json_file, 'w') as f:
    json.dump(generated_test_cases, f, indent=2, default=str)
print(f"✅ Saved JSON: {json_file}")

# 2. Save as CSV
df_data = []
for tc in generated_test_cases:
    if not tc.get('fallback'):
        df_data.append({
            'ID': tc.get('id'),
            'Title': tc.get('title'),
            'Category': tc.get('category'),
            'Priority': tc.get('priority'),
            'Description': tc.get('description'),
            'Preconditions': tc.get('preconditions'),
            'Steps': '; '.join(tc.get('test_steps', [])),
            'Expected Results': tc.get('expected_results'),
            'Test Data': tc.get('test_data'),
            'Automation Feasible': tc.get('automation_feasible'),
            'Duration': tc.get('estimated_duration'),
            'Compliance': ', '.join(tc.get('compliance', []))
        })

if df_data:
    df = pd.DataFrame(df_data)
    csv_file = os.path.join(output_dir, f"test_cases_fixed_{timestamp}.csv")
    df.to_csv(csv_file, index=False)
    print(f"✅ Saved CSV: {csv_file}")
    print(f"   - {len(df_data)} test cases exported")

# 3. Create Markdown report
md_file = os.path.join(output_dir, f"test_report_fixed_{timestamp}.md")
with open(md_file, 'w') as f:
    f.write("# Test Cases Report (FIXED Pipeline)\n\n")
    f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Total Test Cases: {len(generated_test_cases)}\n")
    f.write(f"Successful Generation: {successful}/{len(generated_test_cases)}\n\n")
    
    f.write("## Test Cases\n\n")
    
    for tc in generated_test_cases:
        if not tc.get('fallback'):
            f.write(f"### {tc.get('id')}: {tc.get('title')}\n\n")
            f.write(f"**Category:** {tc.get('category')} | ")
            f.write(f"**Priority:** {tc.get('priority')} | ")
            f.write(f"**Duration:** {tc.get('estimated_duration')}\n\n")
            
            f.write(f"**Description:** {tc.get('description')}\n\n")
            
            if tc.get('preconditions'):
                f.write(f"**Preconditions:** {tc.get('preconditions')}\n\n")
            
            f.write("**Test Steps:**\n")
            for i, step in enumerate(tc.get('test_steps', []), 1):
                f.write(f"{i}. {step}\n")
            
            f.write(f"\n**Expected Results:** {tc.get('expected_results')}\n")
            
            if tc.get('test_data'):
                f.write(f"\n**Test Data:** {tc.get('test_data')}\n")
            
            if tc.get('edge_cases'):
                f.write("\n**Edge Cases:**\n")
                for edge in tc.get('edge_cases'):
                    f.write(f"- {edge}\n")
            
            if tc.get('compliance'):
                f.write(f"\n**Compliance:** {', '.join(tc['compliance'])}\n")
            
            f.write(f"\n**Automation Feasible:** {'Yes' if tc.get('automation_feasible') else 'No'}\n")
            
            f.write("\n---\n\n")

print(f"✅ Saved Markdown Report: {md_file}")

# 4. Display summary
print(f"\n📁 All files saved to: {output_dir}")
print("\n📊 Export Summary:")
print(f"  - JSON: All {len(generated_test_cases)} test cases")
print(f"  - CSV: {len(df_data)} non-fallback test cases")
print(f"  - Markdown: Detailed report with {len(df_data)} test cases")

# Show a sample
if df_data:
    print("\n📋 Sample Test Case from Export:")
    sample = df_data[0]
    for key, value in sample.items():
        if key != 'Steps':  # Skip long field
            print(f"  {key}: {value}")

print("\n💡 TIP: Open the CSV in Excel or import the JSON into your test management tool!")


✅ Saved JSON: /Users/shtlpmac036/Documents/Personal/GenAI Hack /data/generated_test_cases/test_cases_fixed_20250914_231800.json
✅ Saved CSV: /Users/shtlpmac036/Documents/Personal/GenAI Hack /data/generated_test_cases/test_cases_fixed_20250914_231800.csv
   - 8 test cases exported
✅ Saved Markdown Report: /Users/shtlpmac036/Documents/Personal/GenAI Hack /data/generated_test_cases/test_report_fixed_20250914_231800.md

📁 All files saved to: /Users/shtlpmac036/Documents/Personal/GenAI Hack /data/generated_test_cases

📊 Export Summary:
  - JSON: All 8 test cases
  - CSV: 8 non-fallback test cases
  - Markdown: Detailed report with 8 test cases

📋 Sample Test Case from Export:
  ID: TC_001
  Title: Successful User Registration with Valid Data
  Category: Functional
  Priority: High
  Description: Verify successful user registration with all required fields using valid data.
  Preconditions: A clean database with no pre-existing user accounts.  The registration page should be accessible.
  Ex

## 🎮 Interactive: Generate Your Own Test Case

Try the fixed pipeline with your own requirement!


In [ ]:
# 🎮 INTERACTIVE TEST CASE GENERATOR (FIXED VERSION)
print("="*60)
print("🎮 INTERACTIVE TEST CASE GENERATOR")
print("="*60)

# MODIFY THIS REQUIREMENT TO GENERATE YOUR OWN TEST CASE
custom_requirement = """
Test the password complexity validation with the following rules:
- Minimum 8 characters
- At least one uppercase letter  
- At least one number
- At least one special character
- Password should not contain username
"""

print(f"\n📋 Your Requirement:\n{custom_requirement}")
print("\n" + "-"*50)

# Generate test case
print("\n⚡ Generating with FIXED JSON mode...")
custom_test_case = rag_generator.generate_test_case(custom_requirement)

if not custom_test_case.get('fallback'):
    print("✅ Successfully generated with JSON mode!\n")
    
    # Option to save this individual test case
    save_individual = True  # Set to True to save
    
    if save_individual:
        individual_file = os.path.join(output_dir, f"custom_test_case_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        with open(individual_file, 'w') as f:
            json.dump(custom_test_case, f, indent=2, default=str)
        print(f"💾 Saved to: {individual_file}\n")
else:
    print("⚠️ Used fallback generation\n")

# Display the test case
print("📄 GENERATED TEST CASE:")
print("="*60)
print(f"\n🆔 ID: {custom_test_case.get('id')}")
print(f"📝 Title: {custom_test_case.get('title')}")
print(f"🏷️  Category: {custom_test_case.get('category')}")
print(f"⚡ Priority: {custom_test_case.get('priority')}")
print(f"⏱️  Duration: {custom_test_case.get('estimated_duration')}")
print(f"🤖 Automatable: {custom_test_case.get('automation_feasible')}")

print(f"\n📖 Description:\n{custom_test_case.get('description')}")

if custom_test_case.get('preconditions'):
    print(f"\n⚙️  Preconditions:\n{custom_test_case.get('preconditions')}")

print("\n📋 Test Steps:")
for i, step in enumerate(custom_test_case.get('test_steps', []), 1):
    print(f"  {i}. {step}")

print(f"\n✅ Expected Results:\n{custom_test_case.get('expected_results')}")

if custom_test_case.get('test_data'):
    print(f"\n📊 Test Data:\n{custom_test_case.get('test_data')}")

if custom_test_case.get('edge_cases'):
    print("\n⚠️  Edge Cases:")
    for edge in custom_test_case.get('edge_cases', []):
        print(f"  • {edge}")

if custom_test_case.get('compliance'):
    print(f"\n🏥 Compliance: {', '.join(custom_test_case.get('compliance', []))}")

# Quality check
quality = rag_generator.evaluate_generation(custom_test_case)
print(f"\n📊 Quality Score: {quality['overall_score']:.2%}")

print("\n" + "="*60)
print("💡 TIP: Modify the 'custom_requirement' variable above and re-run!")
print("💡 Your test case is automatically saved in the output directory!")


🎮 INTERACTIVE TEST CASE GENERATOR

📋 Your Requirement:

Test the password complexity validation with the following rules:
- Minimum 8 characters
- At least one uppercase letter  
- At least one number
- At least one special character
- Password should not contain username


--------------------------------------------------

⚡ Generating with FIXED JSON mode...
✅ Successfully generated with JSON mode!

💾 Saved to: /Users/shtlpmac036/Documents/Personal/GenAI Hack /data/generated_test_cases/custom_test_case_20250914_231818.json

📄 GENERATED TEST CASE:

🆔 ID: TC_001
📝 Title: Password Complexity Validation
🏷️  Category: Functional
⚡ Priority: High
⏱️  Duration: 15 minutes
🤖 Automatable: True

📖 Description:
Verify password complexity validation with minimum length, uppercase, number, special character, and username exclusion rules.

⚙️  Preconditions:
User is on the registration page.

📋 Test Steps:
  1. Step 1: Enter a password less than 8 characters and attempt registration.
  2. Step 2: